<a href="https://colab.research.google.com/github/Kkamble21/CapX-Stock-Movement-Analysis/blob/main/stock_movement_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw telethon vaderSentiment yfinance scikit-learn nltk pandas


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.2 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=4f3dee1be836385ecdde512b7215e079f25f5e675ee7417b85a7d60d94a6f486
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built pyaes


In [2]:
import praw
import yfinance as yf
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import nltk


In [3]:
# Set up Reddit API credentials
reddit = praw.Reddit(
    client_id='t0qDLcmxWIBbDkHmgPvC8w',  # Your client ID
    client_secret='gu3617SARToyo4RGygg9SwQ7SeRHEQ',  # Your client secret
    user_agent='my_reddit_script_v1'  # A custom user agent
)

# Scrape posts from 'r/stocks' subreddit
subreddit = reddit.subreddit('stocks')
posts = []
for post in subreddit.hot(limit=100):
    posts.append(post.title + " " + post.selftext)

print(posts[:5])  # Display first 5 posts


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



['r/Stocks Daily Discussion & Fundamentals Friday Nov 15, 2024 This is the daily discussion, so anything stocks related is fine, but the theme for today is on fundamentals, but if fundamentals aren\'t your thing then just ignore the theme.\n\nSome helpful day to day links, including news:\n\n* [Finviz](https://finviz.com/quote.ashx?t=spy) for charts, fundamentals, and aggregated news on individual stocks\n* [Bloomberg market news](https://www.bloomberg.com/markets)\n* StreetInsider news:\n  * [Market Check](https://www.streetinsider.com/Market+Check) - Possibly why the market is doing what it\'s doing including sudden spikes/dips\n  * [Reuters aggregated](https://www.streetinsider.com/Reuters) - Global news\n\n-----\n\nMost fundamentals are updated every 3 months due to the fact that corporations release earnings reports every quarter, so traders are always speculating at what those earnings will say, and investors may change the size of their holdings based on those reports.\n\nExpect

In [4]:
# Download stock data using yfinance (Example: Apple stock)
data = yf.download('AAPL', start='2020-01-01', end='2024-01-01')
data.to_csv('apple_stock_data.csv')
print(data.head())  # Display first 5 rows of stock data


[*********************100%***********************]  1 of 1 completed

Price                      Adj Close      Close       High        Low  \
Ticker                          AAPL       AAPL       AAPL       AAPL   
Date                                                                    
2020-01-02 00:00:00+00:00  72.796028  75.087502  75.150002  73.797501   
2020-01-03 00:00:00+00:00  72.088303  74.357498  75.144997  74.125000   
2020-01-06 00:00:00+00:00  72.662712  74.949997  74.989998  73.187500   
2020-01-07 00:00:00+00:00  72.320969  74.597504  75.224998  74.370003   
2020-01-08 00:00:00+00:00  73.484367  75.797501  76.110001  74.290001   

Price                           Open     Volume  
Ticker                          AAPL       AAPL  
Date                                             
2020-01-02 00:00:00+00:00  74.059998  135480400  
2020-01-03 00:00:00+00:00  74.287498  146322800  
2020-01-06 00:00:00+00:00  73.447502  118387200  
2020-01-07 00:00:00+00:00  74.959999  108872000  
2020-01-08 00:00:00+00:00  74.290001  132079200  


In [5]:
# Download NLTK datasets if not already done
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# Clean the Reddit data
cleaned_reddit_posts = [clean_text(post) for post in posts]

# Sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score['compound']

# Get sentiment scores for each Reddit post
reddit_sentiment_scores = [analyze_sentiment(post) for post in cleaned_reddit_posts]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [6]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Clean the Reddit data
cleaned_reddit_posts = [clean_text(post) for post in posts]


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [8]:
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download NLTK datasets if not already done
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# Download the punkt_tab data
nltk.download('punkt_tab') # This line is added to download the missing data


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# Clean the Reddit data
cleaned_reddit_posts = [clean_text(post) for post in posts]

# Sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score['compound']

# Get sentiment scores for each Reddit post
reddit_sentiment_scores = [analyze_sentiment(post) for post in cleaned_reddit_posts]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [11]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download NLTK datasets if not already done
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
# Download the VADER lexicon
nltk.download('vader_lexicon') # This line is added to download the missing lexicon


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# Clean the Reddit data
cleaned_reddit_posts = [clean_text(post) for post in posts]

# Sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score['compound']

# Get sentiment scores for each Reddit post
reddit_sentiment_scores = [analyze_sentiment(post) for post in cleaned_reddit_posts]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
# Create DataFrame for Reddit data
df_reddit = pd.DataFrame({'text': cleaned_reddit_posts, 'sentiment': reddit_sentiment_scores})

# Example target: 1 if positive sentiment, 0 if negative
df_reddit['target'] = df_reddit['sentiment'].apply(lambda x: 1 if x > 0 else 0)

# Example feature: Count of the word 'stock' in each post
df_reddit['stock_mentions'] = df_reddit['text'].apply(lambda x: x.count('stock'))

# Prepare data for machine learning
X_reddit = df_reddit[['sentiment', 'stock_mentions']]
y_reddit = df_reddit['target']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_reddit, y_reddit, test_size=0.2)


In [14]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions on the test data
predictions = model.predict(X_test)


In [15]:
from sklearn.metrics import classification_report

# Evaluate the model's performance
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00        18

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [16]:
import joblib

# Save the trained model for future use
joblib.dump(model, 'stock_movement_model.pkl')


['stock_movement_model.pkl']

In [17]:
# Save the DataFrame with sentiment and stock mentions to CSV
df_reddit.to_csv('reddit_stock_sentiment_data.csv', index=False)


In [18]:
new_data = ['This stock is going up today!']  # New Reddit post
new_data_cleaned = [clean_text(post) for post in new_data]
new_data_sentiment = [analyze_sentiment(post) for post in new_data_cleaned]
new_features = pd.DataFrame({'sentiment': new_data_sentiment, 'stock_mentions': [post.count('stock') for post in new_data_cleaned]})
prediction = model.predict(new_features)
print(prediction)  # Output: 1 for positive sentiment, 0 for negative


[1]


In [19]:
import joblib
joblib.dump(model, 'final_stock_movement_model.pkl')


['final_stock_movement_model.pkl']

In [20]:
import os
os.listdir('/content')


['.config',
 'stock_movement_model.pkl',
 'final_stock_movement_model.pkl',
 'reddit_stock_sentiment_data.csv',
 'apple_stock_data.csv',
 'sample_data']

In [21]:
import joblib
model = joblib.load('/content/final_stock_movement_model.pkl')


In [22]:
!git config --global user.name "Kkamble21"
!git config --global user.email "kshitijk536@gmail.com"


In [23]:
!git clone https://github.com/Kkamble21/CapX-Stock-Movement-Analysis.git


Cloning into 'CapX-Stock-Movement-Analysis'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [24]:
!cp -r /content/your_notebook.ipynb /content/CapX-Stock-Movement-Analysis/


cp: cannot stat '/content/your_notebook.ipynb': No such file or directory
